In [36]:
#import the librarries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
import math
def sigmoid(x):
    try:
        return 1.0/(1.0 + math.exp(-x))
    except:
        return 1.0/(1+abs(x))
            
sigmoid_v=np.vectorize(sigmoid)

In [38]:
import math
def Relu(x):
    if x>=0:
        return x
    else:
        return 0
Relu_v=np.vectorize(Relu)

In [39]:
class ANN:
    def __init__(self):  #class constructor to instialize the variables
        self.weights={}
        self.layer_no=1
        self.prev_size=0
        self.ans={}
        self.delta={}
        self.ERROR=[]
        self.loss=0
        self.cnt=0
        
    def addLayer(self,layer_size,input_size=0,output_size=0):
        if input_size!=0:
            self.weights[self.layer_no]=np.random.uniform(-1.0,1.0, size=(input_size,layer_size))/np.sqrt(2/(layer_size*input_size))

            self.delta[self.layer_no]=np.zeros((input_size,layer_size))        
            self.inputL=input_size
            self.prev_size=layer_size
            self.layer_no+=1
        elif output_size!=0:
            self.weights[self.layer_no]=np.random.uniform(-1.0,1.0,size=(self.prev_size,output_size))/np.sqrt(2/(self.prev_size*output_size))
            self.delta[self.layer_no]=np.zeros((self.prev_size,output_size))
            self.outputL=output_size
            self.layer_no+=1
        else:
            self.weights[self.layer_no]=np.random.uniform(-1.0,1.0,size=(self.prev_size,layer_size))/np.sqrt(2/(self.prev_size*layer_size))
            self.delta[self.layer_no]=np.zeros((self.prev_size,layer_size))
            self.layer_no+=1
            self.prev_size=layer_size
    def printweights(self):
        for i in range(1,self.layer_no):
            print(self.weights[i])
    def forwardprop(self,X,Y):
        self.ans={}
        self.f_no=2
        self.ans[1]=X
        while self.f_no<self.layer_no:
            self.ans[self.f_no]=sigmoid_v(np.dot(self.ans[self.f_no-1],self.weights[self.f_no-1]))
            self.f_no+=1
        self.ans[self.f_no]=sigmoid_v(np.dot(self.ans[self.f_no-1],self.weights[self.f_no-1]))
    def backprop(self,X,Y):
        self.f_no=self.layer_no-1
        self.prev_delta=np.subtract(self.ans[self.f_no+1],Y)
        self.cnt+=np.sum(self.prev_delta)
        self.ERROR.append(np.sum(np.square(self.prev_delta)))
        while self.f_no>=1:
            #print(self.f_no)
            for i in range(self.ans[self.f_no].shape[0]):
                #print(self.prev_delta)
                #print("Vikhyat")
                self.delta[self.f_no][i,:]=self.delta[self.f_no][i,:] + self.prev_delta*self.ans[self.f_no][i]
            self.prev_delta=np.dot(self.prev_delta,(self.weights[self.f_no]).T)
            self.temp=np.ones((1,self.ans[self.f_no].shape[0]))
            self.temp1=np.subtract(self.temp,self.ans[self.f_no])
            self.prev_delta=np.multiply(self.ans[self.f_no],self.temp1)
            self.prev_delta=np.multiply(self.prev_delta,self.temp1)
            self.f_no-=1
    def fit(self,X,Y,batch_size,no_of_epoch,learning_rate):
        self.L=np.zeros((no_of_epoch))
        self.min_loss=1
        for i in range(no_of_epoch):
            self.cnt=0
            for k in range(0,X.shape[0]//batch_size):
                index=np.random.randint(0,X.shape[0],size=(batch_size))
                for j in index:
                    self.forwardprop(X[j],Y[j])
                    self.backprop(X[j],Y[j])
                
                for j in range(1,self.layer_no):
                    self.weights[j]=self.weights[j]*(1-(learning_rate)/X.shape[0]) - (learning_rate*self.delta[j])/batch_size
            print("Loss",abs(self.cnt/Y.shape[0]))
            self.loss=abs(self.cnt)/Y.shape[0]
            self.L[i]=self.loss
            self.min_loss=min(self.min_loss,self.loss)
            
        return self.min_loss
    def plotloss(self):
        plt.plot(self.L)
        plt.show()
    def summary(self):
        l=[]
        for i in range(1,self.layer_no):
            l.append([i ,self.weights[i].shape[0],self.weights[i].shape[1]])
        print(tabulate(l,headers=['Layer_Number','Input_Size','Number_of_Features']))
    def predict(self,X):
        self.predictions=np.zeros((X.shape[0],self.outputL))
        for i in range(X.shape[0]):
            prediction=X[i]
            for j in range(1,self.layer_no-1):
                prediction=sigmoid_v(np.dot(prediction,self.weights[j]))
            prediction=sigmoid_v(np.dot(prediction,self.weights[self.layer_no-1]))
            self.predictions[i]=prediction
        return self.predictions
        
        

In [40]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

c:\users\vikhyat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\vikhyat\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [41]:
n=ANN()

In [42]:
n.addLayer(6,input_size=11,output_size=0)

In [43]:
n.addLayer(6,input_size=0,output_size=0)

In [44]:
n.addLayer(1,input_size=0,output_size=1)

In [45]:
error=n.fit(X_train,y_train,400,20,0.0001)

Loss 0.006951608907657818
Loss 0.002408405860953602
Loss 0.010723112064759942
Loss 0.008450389883872934
Loss 0.004953646882259448
Loss 0.006809209689889826
Loss 0.018205700068198896
Loss 0.014858358531638319
Loss 0.010951377318332787
Loss 0.0184068162341115
Loss 0.015105778485294045
Loss 0.029066890094203428
Loss 0.028621028234645875
Loss 0.01766498660122039
Loss 0.033701966354086045
Loss 0.028321278958735072
Loss 0.019065508338901943
Loss 0.03578973711906504
Loss 0.029516419650194945
Loss 0.035474151592481774


In [46]:
pred=n.predict(X_test)

In [47]:
class confusion_matrix:
    def __init__(self):
        self.pred=np.zeros((2,2))
        
    def cm(self,Y_test,Y_pred):
        for i in range(Y_test.shape[0]):
            if Y_test[i]==1 and Y_pred[i][0]>=0.5:
                self.pred[1][1]+=1
            elif Y_test[i]==0 and Y_pred[i][0]<0.5:
                self.pred[0][0]+=1
            elif Y_test[i]==1 and Y_pred[i][0]<0.5:
                self.pred[1][0]+=1
            elif Y_test[i]==0 and Y_pred[i][0]>=0.5:
                self.pred[0][1]+=1
        return self.pred
            
    

In [48]:
cmatrix=confusion_matrix()

In [49]:
com=cmatrix.cm(y_test,pred)

In [50]:
com

array([[1482.,  113.],
       [ 373.,   32.]])